In [1]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cud

In [2]:
!git clone https://github.com/Efromomr/mamba

%cd /content/mamba

!pip install causal_conv1d --quiet
!pip install . --quiet

Cloning into 'mamba'...
remote: Enumerating objects: 850, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 850 (delta 110), reused 60 (delta 60), pack-reused 727 (from 4)
Receiving objects: 100% (850/850), 1.59 MiB | 2.29 MiB/s, done.
Resolving deltas: 100% (484/484), done.
/content/mamba
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
os.environ["TRITON_F32_DEFAULT"] = "ieee"

In [6]:
import typing as tp
from collections import defaultdict, Counter
import json

import numpy as np
import torch
import torch.nn as nn
import transformers
from transformers import AutoTokenizer
from mamba_ssm.modules.mamba2_simple import Mamba2Simple
import matplotlib.pyplot as plt
def normalize_attn_mat(attn_mat):
    return (attn_mat - torch.min(attn_mat)) / (torch.max(attn_mat) - torch.min(attn_mat))
from datasets import load_dataset
from torch.utils.data import DataLoader

from feedforward import SwiGLU
from model import LanguageModel
from mha import ModifiedMHA

In [7]:
SwiGLU_cfg = {'dim': 64, 'dim_inner': 'null', 'drop_rate': 0, 'bias': False}
Mamba_cfg = {'dim': 64, 'd_state': 16, 'd_conv': 4, 'expand': 2, "headdim": 8, "A_init_range": (1, 1.1), 'compute_attn': True, "use_mem_eff_path": False}
Attn_cfg = {'embed_dim': 64, "num_heads": 16, 'dtype': torch.float, 'batch_first': True }

In [8]:
MODEL_NAME = 'efromomr/mamba_x2_attn_smal'

In [9]:
device = torch.device('cuda')

In [10]:
tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
check_model = LanguageModel.from_pretrained(MODEL_NAME,
                                            layers = [Mamba2Simple, SwiGLU, Mamba2Simple, SwiGLU, Mamba2Simple, SwiGLU, Mamba2Simple, SwiGLU, ModifiedMHA, SwiGLU],
                                            layer_cfgs = [Mamba_cfg, SwiGLU_cfg, Mamba_cfg, SwiGLU_cfg, Mamba_cfg, SwiGLU_cfg, Mamba_cfg, SwiGLU_cfg, Attn_cfg, SwiGLU_cfg], dim=64)
check_model = check_model.eval()

config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/9.02M [00:00<?, ?B/s]

In [12]:
check_model = check_model.to(device)

In [13]:
dataset = load_dataset('eraser-benchmark/movie_rationales', trust_remote_code=True)

README.md:   0%|          | 0.00/6.65k [00:00<?, ?B/s]

movie_rationales.py:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/199 [00:00<?, ? examples/s]

In [15]:
with open('/content/sents.json', 'r') as file:
    sents = json.load(file)

In [16]:
c = Counter([word['pos'] for sent in sents for word in sent])
print({k: v/c.total() for k, v in c.items() if k is not None})

c = Counter([word['synt_tag'] for sent in sents for word in sent])
print({k: v/c.total() for k, v in c.items() if k is not None})

{'PRON': 0.06335028881800155, 'AUX': 0.05886139609736472, 'PART': 0.04033106302267437, 'DET': 0.07723079575825502, 'NOUN': 0.2094031094634595, 'ADJ': 0.08159324079662039, 'VERB': 0.1010719314883467, 'ADV': 0.04825703365232635, 'ADP': 0.0840819610885996, 'SCONJ': 0.014886340776503722, 'PROPN': 0.0626145932120585, 'PUNCT': 0.11311319941374257, 'CCONJ': 0.02681840388539242, 'SYM': 0.0006092479236715809, 'NUM': 0.012133229876138747, 'INTJ': 0.0023967583412363136, 'X': 0.0009598528608788114}
{'expl': 0.0011955053596574418, 'aux': 0.03711239474667356, 'neg': 0.015713998333189642, 'ccomp': 0.024599821823720436, 'det': 0.07636290484811908, 'attr': 0.017122165703939995, 'amod': 0.06549989941661638, 'nsubj': 0.07661580021266201, 'relcl': 0.01334597810156048, 'advmod': 0.050285944190591146, 'dobj': 0.04398655056470385, 'prep': 0.07481104692933299, 'compound': 0.045894761042618616, 'pobj': 0.08288645572894215, 'mark': 0.010782538724602695, 'advcl': 0.015909417478518262, 'conj': 0.04407851251544673

In [19]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [20]:
examples = []

for i, line in enumerate(dataset['test']['review']):
  input_ids = torch.tensor(tokenizer.encode(line, max_length=64, truncation=True))[None, :]
  check_model(input_ids.to(device))

  modules_with_attn = []

  for name, module in check_model.named_modules():
      if isinstance(module, ModifiedMHA) or isinstance(module, Mamba2Simple):
        modules_with_attn.append(module)

  layers = []

  for layer_num, mod in enumerate(modules_with_attn):
    heads = []
    if type(mod) == ModifiedMHA:
        for head_num, head in enumerate(mod.attn_matrix[0]):
          head = normalize_attn_mat(head.cpu().detach())
          heads.append(head)
    else:
        for head_num, head in enumerate(mod.attn_matrix):
          head = normalize_attn_mat(head[0][-1].cpu().detach())
          heads.append(head)
    layers.append(torch.stack(heads))
  examples.append([l.tolist() for l in layers])
  gc.collect()
  torch.cuda.empty_cache()

In [12]:
#row-wise top scores

NUM_LAYERS = 5
NUM_HEADS = 16
NUM_EXAMPLES = 100

for l in range(NUM_LAYERS):
  for h in range(NUM_HEADS):
    for key in ['pos', 'position', 'synt_tag']:
      c = Counter([sents[i][np.argmax(examples[i][l][h]) // len(examples[0][0][0])][key] for i in range(NUM_EXAMPLES)])
      if {k: v/c.total() for k, v in c.items() if k is not None and v/c.total() > 0.3}:
        print(f'layer: {l}, head: {h}, key: {key}')
        print({k: v/c.total() for k, v in c.items() if k is not None and v/c.total() > 0.3})

layer: 0, head: 0, key: synt_tag
{'punct': 0.675}
layer: 0, head: 5, key: pos
{'VERB': 0.6125}
layer: 0, head: 8, key: pos
{'PUNCT': 0.6125}
layer: 0, head: 8, key: synt_tag
{'punct': 0.6125}
layer: 0, head: 13, key: pos
{'PROPN': 0.6125}
layer: 0, head: 14, key: pos
{'PUNCT': 0.6125}
layer: 0, head: 14, key: synt_tag
{'punct': 0.6125}
layer: 0, head: 15, key: pos
{'PROPN': 0.6125}
layer: 1, head: 0, key: pos
{'NOUN': 0.6125}
layer: 1, head: 1, key: pos
{'NOUN': 0.6125}
layer: 1, head: 2, key: pos
{'NOUN': 0.6125}
layer: 1, head: 4, key: pos
{'NOUN': 0.675}
layer: 1, head: 5, key: pos
{'NOUN': 0.675}
layer: 1, head: 6, key: pos
{'PROPN': 0.6125}
layer: 1, head: 8, key: pos
{'NOUN': 0.6125}
layer: 1, head: 9, key: pos
{'NOUN': 0.6125}
layer: 1, head: 11, key: pos
{'NOUN': 0.6125}
layer: 2, head: 4, key: synt_tag
{'nsubj': 0.6125}
layer: 2, head: 12, key: pos
{'VERB': 0.6125}
layer: 2, head: 14, key: pos
{'NOUN': 0.6125}
layer: 2, head: 15, key: pos
{'NOUN': 0.6125}
layer: 3, head: 0, ke

In [14]:
#column-wise top scores

for l in range(NUM_LAYERS):
  for h in range(NUM_HEADS):
    for key in ['pos', 'position', 'synt_tag']:
      c = Counter([sents[i][np.argmax(examples[i][l][h]) % len(examples[0][0][0])][key] for i in range(NUM_EXAMPLES)])
      if {k: v/c.total() for k, v in c.items() if k is not None and v/c.total() > 0.3}:
        print(f'layer: {l}, head: {h}, key: {key}')
        print({k: v/c.total() for k, v in c.items() if k is not None and v/c.total() > 0.3})

layer: 0, head: 0, key: pos
{'DET': 0.6125}
layer: 0, head: 0, key: synt_tag
{'det': 0.6125}
layer: 0, head: 4, key: pos
{'PROPN': 0.6125}
layer: 0, head: 5, key: synt_tag
{'punct': 0.6125}
layer: 0, head: 6, key: pos
{'VERB': 0.6125}
layer: 0, head: 8, key: pos
{'PROPN': 0.7375}
layer: 0, head: 9, key: pos
{'DET': 0.6125}
layer: 0, head: 9, key: synt_tag
{'det': 0.6125}
layer: 0, head: 10, key: pos
{'NOUN': 0.675}
layer: 0, head: 13, key: pos
{'AUX': 0.6125}
layer: 0, head: 15, key: pos
{'PROPN': 0.675}
layer: 1, head: 0, key: pos
{'NOUN': 0.675}
layer: 1, head: 2, key: pos
{'NOUN': 0.6125}
layer: 1, head: 5, key: pos
{'NOUN': 0.7375}
layer: 1, head: 7, key: pos
{'NOUN': 0.6125}
layer: 1, head: 10, key: pos
{'NOUN': 0.6125}
layer: 1, head: 12, key: pos
{'NOUN': 0.675}
layer: 1, head: 13, key: pos
{'NOUN': 0.6125}
layer: 1, head: 14, key: pos
{'DET': 0.8625}
layer: 1, head: 14, key: synt_tag
{'det': 0.8625}
layer: 1, head: 15, key: pos
{'NOUN': 0.6125}
layer: 2, head: 0, key: pos
{'PRO